<a href="https://www.linkedin.com/in/hern%C3%A1n-contigiani-41260679/"><img src="https://raw.githubusercontent.com/hernancontigiani/tf_certification/main/images/hc_banner.png" width="1000" align="center"></a>


# Tensorflow Developer Certification

NLP practice\
Extra: Tokenizer layer inside model (not compatible with save format .h5)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

In [ ]:
!wget https://raw.githubusercontent.com/hernancontigiani/tf_certification/main/utils_tf.py
from utils_tf import *

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
!unzip nlp_getting_started.zip

--2021-12-17 12:52:43--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.191.128, 172.217.219.128, 209.85.147.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.191.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2021-12-17 12:52:43 (51.1 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]

Archive:  nlp_getting_started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [ ]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
df_train["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df_train["text"].to_numpy(), df_train["target"].to_numpy(), test_size=0.1, random_state=42)

In [ ]:
# Fin the average and max sequence

In [ ]:
import numpy as np
vocab = []
for doc in X_train:
    words = doc.split(" ")
    for word in words:
        vocab.append(word)

vocab = set(vocab)

print("Total", len(vocab))
print("Max", max([len(i.split(" ")) for i in X_train]))
print("Average", round(np.mean([len(i.split(" ")) for i in X_train])))

Total 29619
Max 54
Average 15


### Base model Naves

In [ ]:
# baseline TF-IDF Naves
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support

pipe = Pipeline([
                ("tfidf", TfidfVectorizer()),
                 ("cls", MultinomialNB()) 
                ])

pipe.fit(X_train, y_train)
print("Score:", pipe.score(X_valid, y_valid))

m_prediction, m_recall, m_f1, _ = precision_recall_fscore_support(y_valid, pipe.predict(X_valid), average="weighted")
print(m_prediction, m_recall, m_f1)


Score: 0.7782152230971129
0.792992256322435 0.7782152230971129 0.7703527809038113


### Embeddings and Tokenizer

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

In [ ]:
max_vocab_length = 10000 # max numbers of words in the vocabulary
max_length = 15

# max_tokens --> takes de most commons words under that number
# standardize --> to lower case and take of punctation (default)
# split --> charter used for split (default whitespace)
# outputmode --> default as int (numbers, could be binary or TF-IDF)
# output_sequence_length --> None automatically ajust sequence to max long sequence
# pad_to_max_tokens --> padding with 0 to max sequence lenght defined
text_vectorization = TextVectorization(max_tokens=max_vocab_length,
                                       output_sequence_length=max_length,
                                       pad_to_max_tokens=True
                                       )
text_vectorization.adapt(X_train)

In [ ]:
print("Vocab size:", text_vectorization.vocabulary_size())

Vocab size: 10000


In [ ]:
print("Vocab unique words:", len(text_vectorization.get_vocabulary()))

Vocab unique words: 10000


In [ ]:
t1 = "Hi susana"
text_vectorization([t1])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1398,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [ ]:
text_vectorization.get_vocabulary().index("hi")

1398

In [ ]:
text_vectorization.get_vocabulary()[1]

'[UNK]'

In [ ]:
embed_dim = 128

embedding = layers.Embedding(
          input_dim=max_vocab_length,  # Tokenizer max vocab length
          output_dim=embed_dim,  # dimensión de los embeddings utilizados
          input_length=max_length, # max sequence length
          #weights=[embedding_matrix],  # matrix de embeddings
          #trainable=False)      # marcar como layer no entrenable si se define una matrix
          )

In [ ]:
embedding(text_vectorization([t1]))

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01207213,  0.00063944,  0.02580043, ...,  0.01401912,
         -0.01091976, -0.00718091],
        [ 0.03866836, -0.00212086,  0.02003023, ...,  0.03480745,
          0.0236781 ,  0.0226058 ],
        [-0.04812983,  0.02644349, -0.04801338, ..., -0.02599874,
          0.00877376, -0.02361149],
        ...,
        [-0.04812983,  0.02644349, -0.04801338, ..., -0.02599874,
          0.00877376, -0.02361149],
        [-0.04812983,  0.02644349, -0.04801338, ..., -0.02599874,
          0.00877376, -0.02361149],
        [-0.04812983,  0.02644349, -0.04801338, ..., -0.02599874,
          0.00877376, -0.02361149]]], dtype=float32)>

### NLP con DNN + Embeddings

In [ ]:
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorization(inputs)
x = embedding(x)
#x = layers.Flatten()(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs, outputs, name="model_1")

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"]
              )

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-tra

In [ ]:
h = model.fit(X_train.reshape(-1, 1), y_train, validation_data=(X_valid.reshape(-1, 1), y_valid), epochs=2)

Epoch 1/2
215/215 [==============================] - 6s 25ms/step - loss: 0.6108 - accuracy: 0.6901 - val_loss: 0.5428 - val_accuracy: 0.7651
Epoch 2/2
215/215 [==============================] - 5s 23ms/step - loss: 0.4406 - accuracy: 0.8181 - val_loss: 0.4842 - val_accuracy: 0.7966


In [ ]:
y_test_prob = model.predict(X_valid)
print("Prob shape", y_test_prob.shape)
y_test = tf.squeeze(tf.round(y_test_prob))
y_test

Prob shape (762, 1)


<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1.,
       1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0.,
       0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1.,
       1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0

### Obtener los embeddings entrenados

In [ ]:
embeddings_weights = model.get_layer("embedding").get_weights()[0]
embeddings_weights.shape

(10000, 128)

### LSTM

In [ ]:
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorization(inputs)
x = embedding(x)
x = layers.LSTM(units=64, return_sequences=True)(x)
print(x.shape)
x = layers.LSTM(units=64)(x)
print(x.shape) # not more sequence here (15)
x = layers.Dense(65, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model2 = tf.keras.Model(inputs, outputs, name="model_1")

model2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"]
              )

model2.summary()

(None, 15, 64)
(None, 64)
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_8 (LSTM)               (None, 15, 64)            49408     
                                                                 
 lstm_9 (LSTM)               (None, 64)                33024     
                                                                 
 dense_9 (Dense)             (None, 65)                4225      
                                 

In [ ]:
h2 = model2.fit(X_train.reshape(-1, 1), y_train, validation_data=(X_valid.reshape(-1, 1), y_valid), epochs=2)

Epoch 1/2
215/215 [==============================] - 24s 45ms/step - loss: 0.1524 - accuracy: 0.9485 - val_loss: 0.7518 - val_accuracy: 0.7323
Epoch 2/2
215/215 [==============================] - 14s 66ms/step - loss: 0.0866 - accuracy: 0.9660 - val_loss: 1.1490 - val_accuracy: 0.7467


In [ ]:
model2.evaluate(X_valid.reshape(-1, 1), y_valid)[0]

24/24 [==============================] - 0s 13ms/step - loss: 0.7597 - accuracy: 0.7664


0.7597036957740784

### GRU

In [ ]:
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorization(inputs)
x = embedding(x)
#x = layers.GRU(units=64, return_sequences=True)(x)
#print(x.shape)
x = layers.GRU(units=64)(x)
print(x.shape) # not more sequence here (15)
x = layers.Dense(65, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model3 = tf.keras.Model(inputs, outputs, name="model_1")

model3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"]
              )

model3.summary()

(None, 64)
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_2 (GRU)                 (None, 64)                37248     
                                                                 
 dense_13 (Dense)            (None, 65)                4225      
                                                                 
 dense_14 (Dense)            (None, 1)                 66        
                                                

In [ ]:
h3 = model3.fit(X_train.reshape(-1, 1), y_train, validation_data=(X_valid.reshape(-1, 1), y_valid), epochs=2)

Epoch 1/2
215/215 [==============================] - 10s 31ms/step - loss: 0.1486 - accuracy: 0.9460 - val_loss: 0.7578 - val_accuracy: 0.7454
Epoch 2/2
215/215 [==============================] - 6s 29ms/step - loss: 0.0692 - accuracy: 0.9734 - val_loss: 0.9989 - val_accuracy: 0.7480


### LSTM Bidireccional

In [ ]:
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorization(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(units=64, return_sequences=True))(x)
print(x.shape)
x = layers.Bidirectional(layers.LSTM(units=64))(x)
print(x.shape) # not more sequence here (15)
x = layers.Dense(65, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model4 = tf.keras.Model(inputs, outputs, name="model_1")

model4.compile(optimizer=tf.keras.optimizers.Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"]
              )

model4.summary()

(None, 15, 128)
(None, 128)
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                               

In [ ]:
h4 = model4.fit(X_train.reshape(-1, 1), y_train, validation_data=(X_valid.reshape(-1, 1), y_valid), epochs=2)

Epoch 1/2
215/215 [==============================] - 33s 84ms/step - loss: 0.0989 - accuracy: 0.9661 - val_loss: 1.0718 - val_accuracy: 0.7375
Epoch 2/2
215/215 [==============================] - 13s 62ms/step - loss: 0.0546 - accuracy: 0.9769 - val_loss: 1.3740 - val_accuracy: 0.7205


### CONV1

In [ ]:
# one sequence per input
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorization(inputs)
x = embedding(x)

# kernels_size --> it looks 5 words at a time
# strides --> move one word to other (one by one)
# padding --> add padding to match output desired (same keep sequence length, valid minimice)
x = layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="same", activation="relu")(x)
x = layers.GlobalMaxPool1D()(x) # it works better with "MaxPool" to avoid padding (0) influence

#x = layers.Dense(65, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model5 = tf.keras.Model(inputs, outputs, name="model_1")

model5.compile(optimizer=tf.keras.optimizers.Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"]
              )

model5.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 15, 32)            20512     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_19 (Dense)            (None, 1)                 33  

In [ ]:
h5 = model5.fit(X_train.reshape(-1, 1), y_train, validation_data=(X_valid.reshape(-1, 1), y_valid), epochs=2)

Epoch 1/2
215/215 [==============================] - 7s 28ms/step - loss: 0.1498 - accuracy: 0.9561 - val_loss: 0.8975 - val_accuracy: 0.7533
Epoch 2/2
215/215 [==============================] - 6s 26ms/step - loss: 0.0772 - accuracy: 0.9724 - val_loss: 1.0326 - val_accuracy: 0.7310


### Embeddings tranfer learning

Modelos pre entrenados en tensorflow hub:\
https://tfhub.dev/s?module-type=text-classification,text-embedding,text-generation,text-language-model,text-preprocessing,text-question-answering,text-retrieval-question-answering,text-segmentation

In [ ]:
# Embeddings pre entrenados:
# De wikipedia:
# https://tfhub.dev/google/collections/wiki40b-lm/1

# Universal sentence encoder (USE) (512 vectors):
# https://tfhub.dev/google/collections/universal-sentence-encoder/1
# https://tfhub.dev/google/universal-sentence-encoder/4

# large, lite (mobil)

In [ ]:
import tensorflow_hub as hub

base_embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                            input_shape=[], # this model support dynamic input shape
                            dtype=tf.string, # inputs will be strings
                            trainable=False,
                            name="USE"
                            )

In [ ]:
inputs = layers.Input(shape=(), dtype=tf.string)
x = base_embed(inputs)
print(x.shape)

x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model6 = tf.keras.Model(inputs, outputs, name="model_1")

model6.compile(optimizer=tf.keras.optimizers.Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"]
              )

model6.summary()

(None, 512)
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None,)]                 0         
                                                                 
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_25 (Dense)            (None, 64)                32832     
                                                                 
 dense_26 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
h6 = model6.fit(X_train.reshape(-1, 1), y_train, validation_data=(X_valid.reshape(-1, 1), y_valid), epochs=2)

Epoch 1/2
215/215 [==============================] - 8s 24ms/step - loss: 0.5070 - accuracy: 0.7841 - val_loss: 0.4262 - val_accuracy: 0.8058
Epoch 2/2
215/215 [==============================] - 3s 15ms/step - loss: 0.4171 - accuracy: 0.8146 - val_loss: 0.4068 - val_accuracy: 0.8189


In [ ]:
model6.evaluate(X_valid.reshape(-1, 1), y_valid)

24/24 [==============================] - 0s 16ms/step - loss: 0.4068 - accuracy: 0.8189


[0.40684860944747925, 0.8188976645469666]

### TF en 10%

In [ ]:
# ESTO ESTA MAL PORQUE TOMA DATOS QUE SE USARON EN VALID
# df_train_10 = df_train[["text", "target"]].sample(frac=0.1, random_state=42)

# print(df_train_10["target"].value_counts())

# X_train_10, y_train_10 = df_train_10["text"].to_numpy(), df_train_10["target"].to_numpy()

#print(X_train_10.shape)

# ESTO NO HACE FALTA PORQUE LOS DATOS X_TRAIN YA ESTAN MEZCLADOS
# idxs = np.random.choice(range(len(X_train)), size=int(len(X_train)/10))
# X_train_10 = X_train[idxs]
# y_train_10 = y_train[idxs]

idxs = int(len(X_train)/10)
X_train_10 = X_train[:idxs]
y_train_10 = y_train[:idxs]

X_train_10.shape

(685,)

In [ ]:
pd.Series(y_train).value_counts()

0    3916
1    2935
dtype: int64

In [ ]:
pd.Series(y_train_10).value_counts()

0    404
1    281
dtype: int64

In [ ]:
# Create a new model base of arquitecture of model 6, with new layers and weights
model7 = tf.keras.models.clone_model(model6)

model7.compile(optimizer=tf.keras.optimizers.Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"]
              )

model7.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None,)]                 0         
                                                                 
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_25 (Dense)            (None, 64)                32832     
                                                                 
 dense_26 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
h7 = model7.fit(X_train_10.reshape(-1, 1), y_train_10, validation_data=(X_valid.reshape(-1, 1), y_valid), epochs=2)

Epoch 1/2
22/22 [==============================] - 9s 114ms/step - loss: 0.6718 - accuracy: 0.7051 - val_loss: 0.6489 - val_accuracy: 0.7454
Epoch 2/2
22/22 [==============================] - 1s 49ms/step - loss: 0.6040 - accuracy: 0.8117 - val_loss: 0.5897 - val_accuracy: 0.7664


In [ ]:
model7.evaluate(X_valid.reshape(-1, 1), y_valid)

24/24 [==============================] - 0s 14ms/step - loss: 0.5897 - accuracy: 0.7664


[0.5896990299224854, 0.7664042115211487]

### Save and Load a pretrained model

In [ ]:
model6.save("model_6.h5")

In [ ]:
# Load model with custom hub layer
import tensorflow_hub as tensorflow_hub
loaded_model6 = tf.keras.models.load_model("model_6.h5", custom_objects={"KerasLayer": hub.KerasLayer})

In [ ]:
loaded_model6.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None,)]                 0         
                                                                 
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_25 (Dense)            (None, 64)                32832     
                                                                 
 dense_26 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
loaded_model6.evaluate(X_valid.reshape(-1, 1), y_valid)

24/24 [==============================] - 2s 18ms/step - loss: 0.4068 - accuracy: 0.8189


[0.40684860944747925, 0.8188976645469666]

In [ ]:
# Use SaveModel format (tf)
# Allow use to not specify custom layers, this format save all related the model
model6.save("model_6_SavedModel")

INFO:tensorflow:Assets written to: model_6_SavedModel/assets


INFO:tensorflow:Assets written to: model_6_SavedModel/assets


In [ ]:
loaded_model6_SavedModel = tf.keras.models.load_model("model_6_SavedModel")

In [ ]:
loaded_model6_SavedModel.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None,)]                 0         
                                                                 
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_25 (Dense)            (None, 64)                32832     
                                                                 
 dense_26 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
loaded_model6_SavedModel.evaluate(X_valid.reshape(-1, 1), y_valid)

24/24 [==============================] - 1s 17ms/step - loss: 0.4068 - accuracy: 0.8189


[0.40684860944747925, 0.8188976645469666]